In [4]:
##CHROME SERVICE UPDATE##
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
# 
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.close()
 

In [5]:
##CHROMEDRIVER SERVICE CHECK##

from selenium import webdriver
import time
 
driver = webdriver.Chrome()
driver.get("https://google.com")
time.sleep(2)
driver.close()

In [1]:
##trying fake-userAgent library:##
from fake_useragent import UserAgent
ua = UserAgent()

for _ in range(20):
    user_agent = ua.chrome
    print(user_agent)


Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36
Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 OPR/100.0.0.0
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36
Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0

In [27]:
from selenium.webdriver.common.by import By
##single hardcoded part - from query to printed url##
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Step 1: Open Google Scholar and search for the article
query = "Chen, P.-Y. et al. (2018) ‘EAD: Elastic-net attacks to deep neural networks via adversarial examples’, Proceedings of the AAAI Conference on Artificial Intelligence, 32(1). doi:10.1609/aaai.v32i1.11302."
driver = webdriver.Chrome()
driver.get("https://scholar.google.com")
time.sleep(1) # Let the user actually see something!
search_box = driver.find_element(By.NAME,'q')
time.sleep(1) # Let the user actually see something!
search_box.send_keys(query)
time.sleep(1) # Let the user actually see something!
search_box.send_keys(Keys.RETURN)

# Step 2: Find the ".pdf" attribute in search results
pdf_link = driver.find_element(By.CSS_SELECTOR,'a[href*="/article/"]')     
#.find_elements_by_css_selector('a[href$=".pdf"]')

# Step 3: Print the found PDF links
print(pdf_link.get_attribute("href"))

# Close the browser
driver.quit()


https://ojs.aaai.org/index.php/AAAI/article/view/11302/11161


In [ ]:
## using the code iterating through the excel file##

##bug review/TBD:
# -couldnt find urls contains "/pdf/" - FIXED
# -files as above saved as non-pdf files (as default)
# -handle CAPTCHA by changing user-Agent or other leaps.

In [12]:
import random
from selenium.common import NoSuchElementException, StaleElementReferenceException
##First try - from table to download##

import time
import openpyxl
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Load the Excel file
excel_file_path = fr'C:\Users\Simon\Desktop\CBG\ReadingTaskAll.xlsx'
wb = openpyxl.load_workbook(excel_file_path)
sheet = wb.active

# Create an output directory for PDFs
output_dir = "PDFs"
os.makedirs(output_dir, exist_ok=True)

# Step 1: Open Google Scholar and search for each article
options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
# options.add_argument("--headless")  # Run in headless mode
driver = webdriver.Chrome(options)

for row in sheet.iter_rows(min_row=2, values_only=True):
    # Introduce random delays
    delay = random.uniform(2, 7)  # Adjust the range as needed
    time.sleep(delay)
    bibtex = row[10]  # Assuming BibTeX is in column K
    driver.get("https://scholar.google.com")
    time.sleep(1)  # Let the user actually see something!
    search_box = driver.find_element(By.NAME,'q')
    search_box.send_keys(bibtex)
    print("~~~ROW 27 - send keys(bibtex)  DONE~~~")     ##4DEBUG
    # search_box.send_keys(Keys.RETURN)
    print("~~~ROW 29 - send keys(RETURN) - DONE~~~")    ##4DEBUG
    time.sleep(delay)  # Let the user actually see something!

    # Step 2: Find the ".pdf" attribute in search results
    try:
        pdf_link = driver.find_element(By.CSS_SELECTOR,'a[href$=".pdf"]')  
    except NoSuchElementException:
        try:
            print("~~~ROW 46 - inside 'try /pdf/'~~~")    ##4DEBUG
            pdf_link = driver.find_element(By.CSS_SELECTOR,'a[href*="/pdf/"]')     
        except NoSuchElementException:
            print("~~~ROW 49 - inside 'no such element exception, now 'continue' -'~~~")    ##4DEBUG
            continue  # Handle the case where no PDF link is found      
        
    if pdf_link:
        try:
            print(pdf_link.get_attribute("href"))
        except StaleElementReferenceException:
            print(" got StaleElementReferenceException")
            continue           ##couldnt find the file for "/pdf" (second row)
    else:
        print("No PDF link found")

    # Step 3: Download the PDFs and save them to the output directory
    pdf_url = pdf_link.get_attribute("href")
    response = requests.get(pdf_url)
    if response.status_code == 200:
        print("~~~ROW 52 - inside IF statment - DONE~~~")    ##4DEBUG

        # Get the file name from the URL and save it to the output directory
        pdf_filename = os.path.join(output_dir, os.path.basename(pdf_url))
        print("~~~ROW 56 - os.path.join - DONE~~~")    ##4DEBUG

        with open(pdf_filename, "wb") as pdf_file:
            pdf_file.write(response.content)
            print("~~~ROW 61 - Finished writing content - DONE~~~")    ##4DEBUG

# Close the browser
driver.quit()

# Close the Excel file
wb.close()


~~~ROW 27 - send keys(bibtex)  DONE~~~
~~~ROW 29 - send keys(RETURN) - DONE~~~
https://www.usenix.org/system/files/sec23summer_123-zhu-prepub.pdf
~~~ROW 52 - inside IF statment - DONE~~~
~~~ROW 56 - os.path.join - DONE~~~
~~~ROW 61 - Finished writing content - DONE~~~
~~~ROW 27 - send keys(bibtex)  DONE~~~
~~~ROW 29 - send keys(RETURN) - DONE~~~
~~~ROW 46 - inside 'try /pdf/'~~~
https://arxiv.org/pdf/2109.00124
~~~ROW 52 - inside IF statment - DONE~~~
~~~ROW 56 - os.path.join - DONE~~~
~~~ROW 61 - Finished writing content - DONE~~~
~~~ROW 27 - send keys(bibtex)  DONE~~~
~~~ROW 29 - send keys(RETURN) - DONE~~~
http://openaccess.thecvf.com/content/CVPR2022/papers/Zhu_Infrared_Invisible_Clothing_Hiding_From_Infrared_Detectors_at_Multiple_Angles_CVPR_2022_paper.pdf
~~~ROW 52 - inside IF statment - DONE~~~
~~~ROW 56 - os.path.join - DONE~~~
~~~ROW 61 - Finished writing content - DONE~~~
~~~ROW 27 - send keys(bibtex)  DONE~~~
~~~ROW 29 - send keys(RETURN) - DONE~~~
https://openaccess.thecvf.

KeyboardInterrupt: 

In [10]:
###previous WORKING CODE  - now cleaner (methods etc.)###

import random
from selenium.common import NoSuchElementException, StaleElementReferenceException
import time
import openpyxl
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from fake_useragent import UserAgent
import logging
import datetime

isNameDotPDF = True         ##distinguish between urls .../pdf/.. (FALSE) or ....pdf (true)
options = webdriver.ChromeOptions()

def setup_logging():
    global log_file  # Declare log_file as a global variable
    log_folder = "LOG files"
    os.makedirs(log_folder, exist_ok=True)
    
    # Generate a unique log file name with a timestamp
    log_str= f"log_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.log"
    log_file = os.path.join(log_folder+'\\',log_str)
    print(log_file)
    logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s [%(levelname)s] - %(message)s')

def closeLogs():
    global log_file
    if log_file is not None:
        log_file_handler = None
        for handler in logging.getLogger().handlers:
            if isinstance(handler, logging.FileHandler):
                log_file_handler = handler
                break

        if log_file_handler is not None:
            log_file_handler.stream.close()
            logging.getLogger().removeHandler(log_file_handler)
        log_file = None


def onStart():
    ##CHROME SERVICE UPDATE##
    # from selenium import webdriver
    from selenium.webdriver.chrome.service import Service as ChromeService
    from webdriver_manager.chrome import ChromeDriverManager
    import time
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.close()
    
    ##CHROMEDRIVER SERVICE CHECK##
    driver1 = webdriver.Chrome()
    driver1.get("https://google.com")
    time.sleep(2)
    driver1.close()


def set_up_browser():
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    # options.add_argument("--headless")  # Run in headless mode
    return webdriver.Chrome(options)

def randomize_userAgent():
    driver.quit()
    ua = UserAgent(os='windows')
    user_agent = ua.chrome
    uaString = "user-agent="+user_agent
    logging.info(f"!!!changing userAgent: {uaString}")
    options.add_argument(uaString)
    return webdriver.Chrome(options)

def search_article(driver, bibtex):
    driver.get("https://scholar.google.com")
    time.sleep(1)
    search_box = driver.find_element(By.NAME, 'q')
    search_box.send_keys(bibtex)
    # time.sleep(random.uniform(2, 7))
    time.sleep(1)
    
def find_pdf_link(driver):
    global isNameDotPDF  # Use the global keyword to modify the global variable
    try:
        pdf_link = driver.find_element(By.CSS_SELECTOR, 'a[href$=".pdf"]')
        isNameDotPDF = True
    except NoSuchElementException:
        try:
            pdf_link = driver.find_element(By.CSS_SELECTOR, 'a[href*="/pdf"]')
            isNameDotPDF = False
        except NoSuchElementException:
            try:
                pdf_link = driver.find_element(By.CSS_SELECTOR, 'a[href*="/article/"]')
                isNameDotPDF = False
            except NoSuchElementException:
                try:
                    pdf_link = driver.find_element(By.CSS_SELECTOR, 'a[href*="/download"]')
                    isNameDotPDF = False
                except NoSuchElementException:
                    print("couldnt find URL")
                    logging.info("couldnt find URL")
                    return None
    return pdf_link

def download_pdf(pdf_link, output_dir):
    pdf_url = pdf_link.get_attribute("href")
    response = requests.get(pdf_url)
    logging.info(f"\t\t\t\t82 - response code is: {response.status_code}") #DEBUG
    if response.status_code == 200:          ### or response.status_code==418:
        logging.info("\t\t\t\t84-code response valid") #DEBUG
        if isNameDotPDF:
            logging.info("\t\t\t\t86-isNameDotPdf==TRUE") #DEBUG
            pdf_filename = os.path.join(output_dir, os.path.basename(pdf_url))
            print("\t" + pdf_filename)
            logging.info(f"\t{pdf_filename}")
        else:
            logging.info("\t\t\t\t90-isNameDotPdf==FALSE") #DEBUG
            pdf_filename = os.path.join(output_dir, os.path.basename(pdf_url))
            pdf_filename += ".pdf"
            print("\t" + pdf_filename)
            logging.info(f"\t{pdf_filename}")
        with open(pdf_filename, "wb") as pdf_file:
            logging.info("\t\t\t\t95-with open") #DEBUG
            pdf_file.write(response.content)


def RandomDelay():
        delay = random.uniform(2, 12)  # Adjust the range as needed
        logging.info(f"\t\thuman-like delay is: {delay}")
        time.sleep(delay)


def main():
    # onStart()   ##CHROME SERVICE UPDATE&CHECK##
    setup_logging()  # Set up logging with a unique log file
    excel_file_path = fr'C:\Users\Simon\Desktop\CBG\ReadingTaskAll.xlsx'
    wb = openpyxl.load_workbook(excel_file_path)
    sheet = wb.active
    output_dir = "PDFs"
    os.makedirs(output_dir, exist_ok=True)
    global driver
    driver = set_up_browser()
    rowNum = 1
    try:
        for row in sheet.iter_rows(min_row=2, values_only=True):
            rowNum+=1
            if rowNum%7==0:
                driver = randomize_userAgent()
            print("row num: ",rowNum )
            logging.info(f"row num: {rowNum}" )
            bibtex = row[10]
        
            search_article(driver, bibtex)
            pdf_link = find_pdf_link(driver)
    
            if pdf_link:
                print(pdf_link.get_attribute("href"))
                logging.info(pdf_link.get_attribute("href"))
                try:
                    download_pdf(pdf_link, output_dir)
                except StaleElementReferenceException:
                    logging.info("StaleElementReferenceException occurred")
            else:
                logging.info("!!!second try -  simulating pressing ENTER...")
                sb = driver.find_element(By.NAME, 'q')
                sb.send_keys(Keys.ENTER)       #equivilent of enter
                time.sleep(1)
                pdf_link=find_pdf_link(driver)
                if pdf_link:        ##second try
                    print(pdf_link.get_attribute("href"))
                    logging.info(pdf_link.get_attribute("href"))
                    try:
                        download_pdf(pdf_link, output_dir)
                        logging.info("\t\tdownloaded in second try")
                    except StaleElementReferenceException:
                        logging.info("StaleElementReferenceException occurred")
            # RandomDelay()
    except Exception as e:
        # Log any exceptions that cause the run to stop
        logging.error(f"An error occurred: {str(e)}")
    finally:
        closeLogs()
        driver.quit()
        wb.close()

if __name__ == "__main__":
    main()


LOG files\log_2023-10-28_19-52-37.log
row num:  2
https://www.usenix.org/system/files/sec23summer_123-zhu-prepub.pdf
	PDFs\sec23summer_123-zhu-prepub.pdf
row num:  3
